In [421]:
import pandas as pd
import os
import numpy as np
from collections import Counter

In [422]:
dataset = pd.read_csv('/Users/davidmarchenko/Documents/DS440/Box_Office_Calendar.csv')

In [423]:
dataset

,Xref,Exhibition Week,Dist,Title,Director,Cast,Run Time,Rating,IMAX/3D,Genre,...,Year of Release,Month of Release,Week of Release,Reported Gross,Competition,Wide Competition,Number of Expansions,Locations at Widest,Released During Covid?,Day & Date
0,64894,1/6/12,PAR,"Devil Inside, The (2012)",William Brent Bell,"Bonnie Morgan, Fernanda Andrade, Simon Quarterman",83,R,,"Horror, Suspense",...,2012,1,1,53261944.0,1,0,0,2285,0,0
1,65654,1/13/12,UNI,Contraband,Baltasar Kormákur,"Kate Beckinsale, Mark Wahlberg",110,R,,"Suspense, Foreign",...,2012,1,2,66528000.0,3,0,0,2863,0,0
2,67530,1/13/12,DIS,Beauty And The Beast (1991) (re: 2012),"Gary Trousdale, Kirk Wise","Paige O'Hara, Robby Benson",84,G,3D,"Animation, Fantasy",...,2012,1,2,47617067.0,3,0,0,2625,0,0
3,65491,1/13/12,WB,Joyful Noise,Todd Graff,"Dolly Parton, Queen Latifah",118,PG13,,"Drama, Comedy",...,2012,1,2,30932113.0,3,0,0,2735,0,0
4,56574,1/20/12,SNY,Underworld Awakening,"Björn Stein, Måns Mårlind","Kate Beckinsale, Stephen Rea",89,R,"IMAX, 3D","Action, Fantasy",...,2012,1,3,62321039.0,2,1,0,3078,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1153,215653,11/22/24,WB,Untitled WB Event Film (11/22/24),NaN,NaN,,,,",",...,2024,11,47,NaN,1,0,0,0,0,0
1154,99198,12/20/24,20TH,Avatar 3,James Cameron,"Sam Worthington, Sigourney Weaver, Zoe Saldana",,,,"Action, Adventure",...,2024,12,51,NaN,3,0,0,0,0,0
1155,215656,12/20/24,WB,Untitled NL Event Film (12/20/24),NaN,NaN,,,,",",...,2024,12,51,NaN,3,0,0,0,0,0
1156,225786,12/20/24,PAR,Sonic The Hedgehog 3,NaN,NaN,,,,"Animation, Action",...,2024,12,51,NaN,3,0,0,0,0,0


In [424]:
def toList(x,someList):
  x = x.split(',')
  if isinstance(x, list):  
    for name in x:
        if name == None or name == "":
            pass
        else:
            if str(name[0])==" ":
                name = str(name[1:])
            if name!= "":
                someList.append(name)

def unique(list1):
 
    # initialize a null list
    unique_list = []
     
    # traverse for all elements
    for x in list1:
        # check if exists in unique_list or not
        if x not in unique_list:
            unique_list.append(x)
    return unique_list


In [425]:
dataset.rename(columns=lambda x: x.replace('\n',' '), inplace=True)

In [426]:
dataset.Release = pd.to_datetime(dataset['Release'])
dataset = dataset.drop(columns = 'Exhibition Week')

In [427]:
output = dataset[dataset['Release']>'8/15/2022']

In [428]:
list_of_years = list(dataset['Year of Release'].unique())
for year in list_of_years:
    dataset['year_' + str(year)] = dataset['Year of Release'].apply(lambda x: 1 if year == x else 0)
del list_of_years

In [429]:
list_of_months = list(dataset['Month of Release'].unique())
for month in list_of_months:
    dataset['month_' + str(month)] = dataset['Month of Release'].apply(lambda x: 1 if month == x else 0)
del list_of_months

In [430]:
list_of_genres = list()


# dataset['Genre'].apply(lambda x: list_of_genres.append(x))
dataset['Genre'].apply(lambda x: toList(x,list_of_genres) if pd.notna(x) else x)

0       None
1       None
2       None
3       None
4       None
        ... 
1153    None
1154    None
1155    None
1156    None
1157    None
Name: Genre, Length: 1158, dtype: object

In [431]:
for genre in unique(list_of_genres):
    if len(genre) == 0:
        pass
    else:
        dataset['genre_' + str(genre)] = dataset['Genre'].apply(lambda x: 1 if genre in str(x) else 0)

In [432]:
dataset = dataset.drop(columns = 'Genre')

In [433]:
list_of_distributors = list()
dataset['Dist'].apply(lambda x: list_of_distributors.append(x) if (pd.notna(x)) else x)

for distributor in unique(list_of_distributors):
    dataset['distributor_' + str(distributor)] = dataset['Dist'].apply(lambda x: 1 if distributor==str(x) else 0)

dataset = dataset.drop(columns = 'Dist')

In [434]:
list_of_ratings = list()
dataset['Rating'].apply(lambda x: list_of_ratings.append(x) if pd.notna(x) else x)

for rating in unique(list_of_ratings):
    dataset['rating_' + str(rating)] = dataset['Rating'].apply(lambda x: 1 if rating==str(x) else 0)

dataset = dataset.drop(columns = ['Rating','rating_ '])

In [435]:
list_of_formats = list()

dataset['IMAX/3D'].apply(lambda x: toList(x,list_of_formats) if pd.notna(x) else x)

for format in unique(list_of_formats):
    dataset['format_' + format] = dataset['IMAX/3D'].apply(lambda x: 1 if format in str(x) else 0)

dataset = dataset.drop(columns = "IMAX/3D")

In [436]:
list_of_directors = list()
dataset['Director'].apply(lambda x: list_of_directors.append(x))

c = Counter(list_of_directors)
top_directors = [m[0] for m in c.most_common(20)]

for director in unique(top_directors):
    dataset['directedBy_' + str(director)] = dataset['Director'].apply(lambda x: 1 if (x == director) else 0)

dataset = dataset.drop(columns = ["Director",'Producer(s)'])

In [437]:
list_of_actors = list()

dataset['Cast'].apply(lambda x: toList(x,list_of_actors) if pd.notna(x) else x)

c = Counter(list_of_actors)
top_actors = [m[0] for m in c.most_common(50)]

for actor in top_actors:
    dataset['has_' + actor] = dataset['Cast'].apply(lambda x: 1 if actor in str(x) else 0)

dataset = dataset.drop(columns = "Cast")

In [438]:
list_of_companies = list()

dataset['Production Company'].apply(lambda x: toList(x,list_of_companies) if pd.notna(x) else x)

c = Counter(list_of_companies)
top_companies = [m[0] for m in c.most_common(30)]

for company in top_companies:
    dataset['Studio_' + company] = dataset['Production Company'].apply(lambda x: 1 if company in str(x) else 0)

dataset = dataset.drop(columns = "Production Company")

In [439]:
top_companies

['DENTSU Music And Entertainment',
 'Columbia Pictures',
 'Marvel Studios',
 'Blumhouse Productions',
 'Legendary Entertainment',
 'Screen Gems',
 'Metro-Goldwyn-Mayer (MGM)',
 'DreamWorks Animation',
 'Working Title Films',
 'DC Entertainment',
 'Original Film',
 'Di Bonaventura Pictures',
 'New Line Cinema',
 'Chernin Entertainment',
 'Amblin Entertainment',
 'Skydance Productions',
 'Summit Entertainment',
 'Sony Pictures Animation',
 'Illumination Entertainment',
 'Village Roadshow Pictures',
 'Pixar Animation Studios',
 'Platinum Dunes',
 'Atlas Entertainment',
 'Vertigo Entertainment',
 'Walt Disney Feature Animation',
 'Scott Free Productions',
 'Marc Platt Productions',
 'Inc.',
 'Will Packer Productions',
 'Alcon Entertainment']

In [440]:
list_of_patterns = list()

dataset['Pattern'].apply(lambda x: toList(x,list_of_patterns) if pd.notna(x) else x)

for pattern in unique(list_of_patterns):
    dataset['pattern_' + pattern] = dataset['Pattern'].apply(lambda x: 1 if format in str(x) else 0)

dataset = dataset.drop(columns = "Pattern")

In [441]:
dataset

,Xref,Title,Run Time,Locations at Opening,Synopsis,Release,Year of Release,Month of Release,Week of Release,Reported Gross,...,pattern_IMAX,pattern_Academy Qualifying Run,pattern_LA,pattern_Moderate (IMAX 3D/PLF 3D),pattern_Limited (IMAX 3D,pattern_PLF 3D),pattern_NY/LA in 3D,pattern_NY/LA/BOS,pattern_Kid Shows,pattern_(re)
0,64894,"Devil Inside, The (2012)",83,2285,While trying to help her institutionalized mot...,2012-01-06,2012,1,1,53261944.0,...,0,0,0,0,0,0,0,0,0,0
1,65654,Contraband,110,2863,A security guard with financial troubles consi...,2012-01-13,2012,1,2,66528000.0,...,0,0,0,0,0,0,0,0,0,0
2,67530,Beauty And The Beast (1991) (re: 2012),84,2625,Walt Disney Animation Studios' magical classic...,2012-01-13,2012,1,2,47617067.0,...,0,0,0,0,0,0,0,0,0,0
3,65491,Joyful Noise,118,2735,Joyful Noise tells the story of an unlikely pa...,2012-01-13,2012,1,2,30932113.0,...,0,0,0,0,0,0,0,0,0,0
4,56574,Underworld Awakening,89,3078,UNDERWORLD AWAKENING brings a stunning new dim...,2012-01-20,2012,1,3,62321039.0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1153,215653,Untitled WB Event Film (11/22/24),,,,2024-11-22,2024,11,47,NaN,...,0,0,0,0,0,0,0,0,0,0
1154,99198,Avatar 3,,,,2024-12-20,2024,12,51,NaN,...,0,0,0,0,0,0,0,0,0,0
1155,215656,Untitled NL Event Film (12/20/24),,,,2024-12-20,2024,12,51,NaN,...,0,0,0,0,0,0,0,0,0,0
1156,225786,Sonic The Hedgehog 3,,,,2024-12-20,2024,12,51,NaN,...,0,0,0,0,0,0,0,0,0,0


In [442]:
dataset = dataset.drop(columns = ['Xref','Title','Synopsis'])

In [443]:
dataset['Run Time'] = pd.to_numeric(dataset['Run Time'],errors='coerce')

In [444]:
dataset['Locations at Opening'] = pd.to_numeric(dataset['Locations at Opening'],errors='coerce')

In [445]:
dataset

,Run Time,Locations at Opening,Release,Year of Release,Month of Release,Week of Release,Reported Gross,Competition,Wide Competition,Number of Expansions,...,pattern_IMAX,pattern_Academy Qualifying Run,pattern_LA,pattern_Moderate (IMAX 3D/PLF 3D),pattern_Limited (IMAX 3D,pattern_PLF 3D),pattern_NY/LA in 3D,pattern_NY/LA/BOS,pattern_Kid Shows,pattern_(re)
0,83.0,2285.0,2012-01-06,2012,1,1,53261944.0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,110.0,2863.0,2012-01-13,2012,1,2,66528000.0,3,0,0,...,0,0,0,0,0,0,0,0,0,0
2,84.0,2625.0,2012-01-13,2012,1,2,47617067.0,3,0,0,...,0,0,0,0,0,0,0,0,0,0
3,118.0,2735.0,2012-01-13,2012,1,2,30932113.0,3,0,0,...,0,0,0,0,0,0,0,0,0,0
4,89.0,3078.0,2012-01-20,2012,1,3,62321039.0,2,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1153,NaN,NaN,2024-11-22,2024,11,47,NaN,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1154,NaN,NaN,2024-12-20,2024,12,51,NaN,3,0,0,...,0,0,0,0,0,0,0,0,0,0
1155,NaN,NaN,2024-12-20,2024,12,51,NaN,3,0,0,...,0,0,0,0,0,0,0,0,0,0
1156,NaN,NaN,2024-12-20,2024,12,51,NaN,3,0,0,...,0,0,0,0,0,0,0,0,0,0


In [446]:
# Move Window Gross column to the End

temp_cols=dataset.columns.tolist()
index=dataset.columns.get_loc("Reported Gross")
new_cols=temp_cols[0:index] + temp_cols[index+1:] +temp_cols[index:index+1]
dataset=dataset[new_cols]

In [447]:
dataset

,Run Time,Locations at Opening,Release,Year of Release,Month of Release,Week of Release,Competition,Wide Competition,Number of Expansions,Locations at Widest,...,pattern_Academy Qualifying Run,pattern_LA,pattern_Moderate (IMAX 3D/PLF 3D),pattern_Limited (IMAX 3D,pattern_PLF 3D),pattern_NY/LA in 3D,pattern_NY/LA/BOS,pattern_Kid Shows,pattern_(re),Reported Gross
0,83.0,2285.0,2012-01-06,2012,1,1,1,0,0,2285,...,0,0,0,0,0,0,0,0,0,53261944.0
1,110.0,2863.0,2012-01-13,2012,1,2,3,0,0,2863,...,0,0,0,0,0,0,0,0,0,66528000.0
2,84.0,2625.0,2012-01-13,2012,1,2,3,0,0,2625,...,0,0,0,0,0,0,0,0,0,47617067.0
3,118.0,2735.0,2012-01-13,2012,1,2,3,0,0,2735,...,0,0,0,0,0,0,0,0,0,30932113.0
4,89.0,3078.0,2012-01-20,2012,1,3,2,1,0,3078,...,0,0,0,0,0,0,0,0,0,62321039.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1153,NaN,NaN,2024-11-22,2024,11,47,1,0,0,0,...,0,0,0,0,0,0,0,0,0,NaN
1154,NaN,NaN,2024-12-20,2024,12,51,3,0,0,0,...,0,0,0,0,0,0,0,0,0,NaN
1155,NaN,NaN,2024-12-20,2024,12,51,3,0,0,0,...,0,0,0,0,0,0,0,0,0,NaN
1156,NaN,NaN,2024-12-20,2024,12,51,3,0,0,0,...,0,0,0,0,0,0,0,0,0,NaN


In [448]:
is_target = dataset['Release']>="8/15/22"
isnot_target = dataset['Release']<"8/15/22"
target_ML = dataset[is_target]
final_dataset = dataset[isnot_target]
train = final_dataset.drop(columns=['Release'])
test = target_ML.drop(columns=['Release'])
del final_dataset

In [449]:
train

,Run Time,Locations at Opening,Year of Release,Month of Release,Week of Release,Competition,Wide Competition,Number of Expansions,Locations at Widest,Released During Covid?,...,pattern_Academy Qualifying Run,pattern_LA,pattern_Moderate (IMAX 3D/PLF 3D),pattern_Limited (IMAX 3D,pattern_PLF 3D),pattern_NY/LA in 3D,pattern_NY/LA/BOS,pattern_Kid Shows,pattern_(re),Reported Gross
0,83.0,2285.0,2012,1,1,1,0,0,2285,0,...,0,0,0,0,0,0,0,0,0,53261944.0
1,110.0,2863.0,2012,1,2,3,0,0,2863,0,...,0,0,0,0,0,0,0,0,0,66528000.0
2,84.0,2625.0,2012,1,2,3,0,0,2625,0,...,0,0,0,0,0,0,0,0,0,47617067.0
3,118.0,2735.0,2012,1,2,3,0,0,2735,0,...,0,0,0,0,0,0,0,0,0,30932113.0
4,89.0,3078.0,2012,1,3,2,1,0,3078,0,...,0,0,0,0,0,0,0,0,0,62321039.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999,105.0,4314.0,2022,7,31,1,1,0,4314,0,...,0,0,0,0,0,0,0,0,0,75356058.0
1000,126.0,4357.0,2022,8,32,2,0,0,4357,0,...,0,0,0,0,0,0,0,0,0,80532558.0
1001,96.0,3175.0,2022,8,32,2,0,0,3175,0,...,0,0,0,0,0,0,0,0,0,12660865.0
1002,107.0,1548.0,2022,8,33,2,0,0,1548,0,...,0,0,0,0,0,0,0,0,0,6104420.0


In [450]:
test

,Run Time,Locations at Opening,Year of Release,Month of Release,Week of Release,Competition,Wide Competition,Number of Expansions,Locations at Widest,Released During Covid?,...,pattern_Academy Qualifying Run,pattern_LA,pattern_Moderate (IMAX 3D/PLF 3D),pattern_Limited (IMAX 3D,pattern_PLF 3D),pattern_NY/LA in 3D,pattern_NY/LA/BOS,pattern_Kid Shows,pattern_(re),Reported Gross
1004,93.0,3743.0,2022,8,34,2,0,0,3743,0,...,0,0,0,0,0,0,0,0,0,21779575.0
1005,99.0,498.0,2022,8,34,2,0,0,498,0,...,0,0,0,0,0,0,0,0,0,3688662.0
1006,105.0,3114.0,2022,8,35,2,0,0,3114,0,...,0,0,0,0,0,0,0,0,0,9044963.0
1007,124.0,1246.0,2022,9,36,2,0,0,1246,0,...,0,0,0,0,0,0,0,0,0,NaN
1008,103.0,2200.0,2022,9,37,1,0,0,2200,0,...,0,0,0,0,0,0,0,0,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1153,NaN,NaN,2024,11,47,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,NaN
1154,NaN,NaN,2024,12,51,3,0,0,0,0,...,0,0,0,0,0,0,0,0,0,NaN
1155,NaN,NaN,2024,12,51,3,0,0,0,0,...,0,0,0,0,0,0,0,0,0,NaN
1156,NaN,NaN,2024,12,51,3,0,0,0,0,...,0,0,0,0,0,0,0,0,0,NaN


In [451]:
train_X, train_y = train.iloc[:, :-1], train.iloc[:, -1]

test_X, test_y = test.iloc[:, :-1], test.iloc[:, -1]

In [452]:
train_X['Run Time']=train_X['Run Time'].fillna(train_X['Run Time'].mean())

In [453]:
train_X['Locations at Opening']=train_X['Locations at Opening'].fillna(train_X['Locations at Opening'].mean())

In [455]:
train_X

,Run Time,Locations at Opening,Year of Release,Month of Release,Week of Release,Competition,Wide Competition,Number of Expansions,Locations at Widest,Released During Covid?,...,pattern_IMAX,pattern_Academy Qualifying Run,pattern_LA,pattern_Moderate (IMAX 3D/PLF 3D),pattern_Limited (IMAX 3D,pattern_PLF 3D),pattern_NY/LA in 3D,pattern_NY/LA/BOS,pattern_Kid Shows,pattern_(re)
0,83.0,2285.0,2012,1,1,1,0,0,2285,0,...,0,0,0,0,0,0,0,0,0,0
1,110.0,2863.0,2012,1,2,3,0,0,2863,0,...,0,0,0,0,0,0,0,0,0,0
2,84.0,2625.0,2012,1,2,3,0,0,2625,0,...,0,0,0,0,0,0,0,0,0,0
3,118.0,2735.0,2012,1,2,3,0,0,2735,0,...,0,0,0,0,0,0,0,0,0,0
4,89.0,3078.0,2012,1,3,2,1,0,3078,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999,105.0,4314.0,2022,7,31,1,1,0,4314,0,...,0,0,0,0,0,0,0,0,0,0
1000,126.0,4357.0,2022,8,32,2,0,0,4357,0,...,0,0,0,0,0,0,0,0,0,0
1001,96.0,3175.0,2022,8,32,2,0,0,3175,0,...,0,0,0,0,0,0,0,0,0,0
1002,107.0,1548.0,2022,8,33,2,0,0,1548,0,...,0,0,0,0,0,0,0,0,0,0


In [456]:
from sklearn.linear_model import LinearRegression

In [457]:
model = LinearRegression().fit(train_X, train_y)

In [458]:
model

LinearRegression()

In [460]:
r_sq = model.score(train_X, train_y)
print(f"coefficient of determination: {r_sq}")

coefficient of determination: 0.6639148554297214


In [463]:
!pip install eli5

     |████████████████████████████████| 216 kB 899 kB/s eta 0:00:01
     |████████████████████████████████| 133 kB 4.8 MB/s eta 0:00:01
     |████████████████████████████████| 47 kB 19.9 MB/s eta 0:00:01
  Created wheel for eli5: filename=eli5-0.13.0-py2.py3-none-any.whl size=107748 sha256=a2d20bbc1019c053aa5c8a597db7116005fe0b87f4cd167aca4b53bd9d7a920f
  Stored in directory: /Users/davidmarchenko/Library/Caches/pip/wheels/7b/26/a5/8460416695a992a2966b41caa5338e5e7fcea98c9d032d055c
Successfully built eli5
  Attempting uninstall: MarkupSafe
    Found existing installation: MarkupSafe 1.1.1
    Uninstalling MarkupSafe-1.1.1:
      Successfully uninstalled MarkupSafe-1.1.1
  Attempting uninstall: jinja2
    Found existing installation: Jinja2 2.11.3
    Uninstalling Jinja2-2.11.3:
      Successfully uninstalled Jinja2-2.11.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency co

In [464]:
import eli5

In [465]:
eli5.show_weights(model)

Weight?,Feature
+29310542600588189696.000,<BIAS>
+212639542735337280.000,x24
+174664197076776480.000,x25
+136688851395138528.000,x26
+98713505722424464.000,x27
+72960140299890720.000,x21
+60738160043389496.000,x28
+58303445608376792.000,x20
+43646750924613312.000,x19
+37975345676465840.000,x3
